In [5]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk, scrolledtext
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression

In [6]:
# =========================
# 1) DATA GENERATION
# =========================

RANDOM_STATE = 42

def create_dataset():
    # Classification base
    X, y_class = make_classification(
        n_samples=800,
        n_features=6,
        n_informative=4,
        n_redundant=0,
        n_repeated=0,
        n_classes=2,
        class_sep=1.5,
        flip_y=0.03,
        random_state=RANDOM_STATE
    )

    feature_names = [f"feat_{i}" for i in range(X.shape[1])]
    df = pd.DataFrame(X, columns=feature_names)
    df["churn"] = y_class

    # Nonlinear continuous target
    np.random.seed(RANDOM_STATE)
    base_signal = (
        50
        + 10 * df["feat_0"]
        - 5 * (df["feat_1"] ** 2)
        + 3 * df["feat_2"] * df["feat_3"]
    )
    noise = np.random.normal(loc=0, scale=10, size=len(df))
    df["monthly_revenue"] = base_signal + noise

    return df, feature_names

df, feature_names = create_dataset()

X = df[feature_names].values
y_cls = df["churn"].values
y_reg = df["monthly_revenue"].values

X_train, X_test, y_cls_train, y_cls_test, y_reg_train, y_reg_test = train_test_split(
    X, y_cls, y_reg,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y_cls
)


In [7]:
# =========================
# 2) MODEL TRAINING FUNCTIONS
# =========================

def run_decision_tree():
    tree_clf = DecisionTreeClassifier(
        max_depth=4,
        min_samples_split=10,
        random_state=RANDOM_STATE
    )
    tree_clf.fit(X_train, y_cls_train)
    y_pred = tree_clf.predict(X_test)

    acc = accuracy_score(y_cls_test, y_pred)
    cm = confusion_matrix(y_cls_test, y_pred)
    report = classification_report(y_cls_test, y_pred)

    result = []
    result.append("=== Decision Tree Classifier ===")
    result.append(f"Accuracy: {acc:.4f}")
    result.append("Confusion Matrix:")
    result.append(str(cm))
    result.append("Classification Report:")
    result.append(report)
    return "\n".join(result)

def run_logistic_linear():
    pipe = Pipeline(steps=[
        ("scaler", StandardScaler()),
        ("log_reg", LogisticRegression(
            solver="lbfgs", max_iter=1000, random_state=RANDOM_STATE
        ))
    ])
    pipe.fit(X_train, y_cls_train)
    y_pred = pipe.predict(X_test)

    acc = accuracy_score(y_cls_test, y_pred)
    cm = confusion_matrix(y_cls_test, y_pred)
    report = classification_report(y_cls_test, y_pred)

    result = []
    result.append("=== Logistic Regression (linear) ===")
    result.append(f"Accuracy: {acc:.4f}")
    result.append("Confusion Matrix:")
    result.append(str(cm))
    result.append("Classification Report:")
    result.append(report)
    return "\n".join(result)

def run_logistic_poly(degree=2):
    pipe = Pipeline(steps=[
        ("scaler", StandardScaler()),
        ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
        ("log_reg", LogisticRegression(
            solver="lbfgs", max_iter=1000, random_state=RANDOM_STATE
        ))
    ])
    pipe.fit(X_train, y_cls_train)
    y_pred = pipe.predict(X_test)

    acc = accuracy_score(y_cls_test, y_pred)
    cm = confusion_matrix(y_cls_test, y_pred)
    report = classification_report(y_cls_test, y_pred)

    result = []
    result.append(f"=== Logistic Regression with Polynomial Features (degree={degree}) ===")
    result.append(f"Accuracy: {acc:.4f}")
    result.append("Confusion Matrix:")
    result.append(str(cm))
    result.append("Classification Report:")
    result.append(report)
    return "\n".join(result)

def run_poly_regression(degree=3):
    pipe = Pipeline(steps=[
        ("scaler", StandardScaler()),
        ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
        ("lin_reg", LinearRegression())
    ])
    pipe.fit(X_train, y_reg_train)
    y_pred = pipe.predict(X_test)

    mse = mean_squared_error(y_reg_test, y_pred)
    r2 = r2_score(y_reg_test, y_pred)

    result = []
    result.append(f"=== Polynomial Regression (degree={degree}) ===")
    result.append(f"MSE: {mse:.4f}")
    result.append(f"R²: {r2:.4f}")
    return "\n".join(result)


In [8]:
# =========================
# 3) TKINTER GUI
# =========================

def on_run_clicked():
    model_choice = model_var.get()
    degree_text = degree_entry.get().strip()

    # default polynomial degree if empty/invalid
    try:
        degree = int(degree_text) if degree_text else 2
    except ValueError:
        degree = 2

    if model_choice == "Decision Tree (Classification)":
        output = run_decision_tree()
    elif model_choice == "Logistic Regression (Classification)":
        output = run_logistic_linear()
    elif model_choice == "Polynomial Logistic (Classification)":
        output = run_logistic_poly(degree=degree)
    elif model_choice == "Polynomial Regression (Regression)":
        output = run_poly_regression(degree=degree)
    else:
        output = "Please select a valid model."

    result_box.config(state="normal")
    result_box.delete(1.0, tk.END)
    result_box.insert(tk.END, output)
    result_box.config(state="disabled")

# Create main window
root = tk.Tk()
root.title("ML Project: Tree, Logistic & Polynomial Regression")

root.geometry("850x600")

# Frame for controls
controls_frame = ttk.Frame(root, padding=10)
controls_frame.pack(side=tk.TOP, fill=tk.X)

# Model selection
ttk.Label(controls_frame, text="Select Model:").grid(row=0, column=0, sticky=tk.W, padx=5, pady=5)

model_var = tk.StringVar()
model_combo = ttk.Combobox(
    controls_frame,
    textvariable=model_var,
    state="readonly",
    width=40,
    values=[
        "Decision Tree (Classification)",
        "Logistic Regression (Classification)",
        "Polynomial Logistic (Classification)",
        "Polynomial Regression (Regression)"
    ]
)
model_combo.grid(row=0, column=1, sticky=tk.W, padx=5, pady=5)
model_combo.current(0)

# Polynomial degree
ttk.Label(controls_frame, text="Polynomial degree (for poly models):").grid(
    row=1, column=0, sticky=tk.W, padx=5, pady=5
)
degree_entry = ttk.Entry(controls_frame, width=10)
degree_entry.grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)
degree_entry.insert(0, "2")

# Run button
run_button = ttk.Button(controls_frame, text="Train & Evaluate", command=on_run_clicked)
run_button.grid(row=0, column=2, rowspan=2, padx=10, pady=5)

# Frame for results
result_frame = ttk.Frame(root, padding=10)
result_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

ttk.Label(result_frame, text="Results:").pack(anchor=tk.W)

result_box = scrolledtext.ScrolledText(result_frame, wrap=tk.WORD, width=100, height=25)
result_box.pack(fill=tk.BOTH, expand=True)
result_box.config(state="disabled")

root.mainloop()